### Install Cellpose-SAM


In [ ]:
!pip install git+https://www.github.com/mouseland/cellpose.git

Check GPU and instantiate model - will download weights.

In [1]:
import numpy as np
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import trange
import matplotlib.pyplot as plt

io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

model = models.CellposeModel(gpu=True)

Input directory with your images (if you have them, otherwise use sample images):

In [35]:
from pathlib import Path
import re
import nibabel as nib
import numpy as np
from tifffile import imread
from cellpose import models, io, train

image_dirs = [
    Path("/scratch/msa6093/Daphnia392/XLayers-1000-2000"),
    Path("/scratch/msa6093/Daphnia392/ZLayer-200")
]
mask_dir = Path("/scratch/msa6093/Daphnia392/CuratedTrainingData")

# Sanity check
for d in image_dirs + [mask_dir]:
    if not d.exists():
        raise FileNotFoundError(f"Directory not found: {d}")

mask_ext = ".nii.gz"

# regex to pull axis and layer from folder name, e.g. "ZLayer-200" → ("Z", "200")
dir_re   = re.compile(r'^([XZ])Layers?-?(\d+)$', re.IGNORECASE)
# regex to pull slice index from filename stem, e.g. "..._28" → "28"
slice_re = re.compile(r'_(\d+)$')

img_paths = []
mask_paths = []

for img_dir in image_dirs:
    dir_match = dir_re.match(img_dir.name)
    if dir_match:
        axis  = dir_match.group(1).upper()
        layer = int(dir_match.group(2))
    else:
        axis = layer = None

    for img_f in sorted(img_dir.glob("*.tif")):
        if axis is not None:
            sm = slice_re.search(img_f.stem)
            if not sm:
                pri


## Train new model

In [ ]:
from pathlib import Path
import re
import nibabel as nib
import numpy as np
from tifffile import imread
from cellpose import models, train

image_dirs = [
    Path("/scratch/msa6093/Daphnia392/XLayers-1000-2000"),
    Path("/scratch/msa6093/Daphnia392/ZLayer-200")
]
mask_dir = Path("/scratch/msa6093/Daphnia392/CuratedTrainingData")
mask_ext = ".nii.gz"

# regex: capture “X” or “Z” then “Layer” or “Layers”, then everything after the dash
dir_re   = re.compile(r'^([XZ])Layers?-?(.+)$', re.IGNORECASE)
slice_re = re.compile(r'_(\d+)$')  # captures the trailing _<slice> in the filename

img_paths, mask_paths = [], []
for img_dir in image_dirs:
    m = dir_re.match(img_dir.name)
    if not m:
        print(f"skipping dir `{img_dir.name}` (didn't match axis+layer)")
        continue

    axis, layer = m.group(1).upper(), m.group(2)  # layer is now e.g. "200" or "1000-2000"
    for tif in sorted(img_dir.glob("*.tif")):
        sm = slice_re.search(tif.stem)
        if not sm:
            print(f"skipping `{tif.name}` (no _<slice> to parse)")
            continue
        slice_idx = sm.group(1)
        mask_name = f"predicted-{axis}{layer}-{slice_idx}{mask_ext}"
        mask_file = mask_dir / mask_name
        if mask_file.exists():
            img_paths.append(tif)
            mask_paths.append(mask_file)
        else:
            print(f"⏭  no mask for {tif.name} → expected `{mask_name}`")

if not img_paths:
    raise RuntimeError("No image↔mask pairs found. Check your folder names & mask naming.")

print(f"Found {len(img_paths)} pairs")

# load images with tifffile, masks with nibabel
train_data   = [imread(str(p))                            for p in img_paths]
train_labels = [nib.load(str(p)).get_fdata().astype(np.uint8) for p in mask_paths]

# training
model = models.CellposeModel(gpu=True)
new_model_path, train_losses, test_losses = train.train_seg(
    model.net,
    train_data      = train_data,
    train_labels    = train_labels,
    batch_size      = 1,
    n_epochs        = 100,
    learning_rate   = 1e-5,
    weight_decay    = 0.1,
    nimg_per_epoch  = max(2, len(train_data)),
    model_name      = "CuratedTrainingData_May30_2Axes",
    min_train_masks = 1
)

print(f"Training complete, model saved to: {new_model_path}")


## Evaluate on test data (optional)

If you have test data, check performance

In [4]:
import numpy as np
from pathlib import Path
from tifffile import imread, imwrite
import nibabel as nib
from cellpose import models, metrics


# path to the folder containing test images
test_dir = Path("/scratch/msa6093/OverlappingPatches-Y")

# (optional) path to the folder containing ground‐truth masks
gt_dir   = Path("/path/to/your/gt_masks")

# where to save the predicted masks
out_dir  = test_dir / "/scratch/msa6093/Full_Masks_Overlapping_Y"

# trained Cellpose model file
trained_model = "/scratch/msa6093/Daphnia392/models/CuratedTrainingData_May28_2Axes"




# make sure output directory exists
out_dir.mkdir(exist_ok=True, parents=True)

# gather all image files in test_dir
img_exts = {".tif", ".tiff", ".png", ".jpg", ".jpeg"}
nii_exts = (".nii", ".nii.gz")

# gather all image files in test_dir
img_files = sorted([
    f for f in test_dir.iterdir()
    if f.suffix.lower() in img_exts
       or f.name.lower().endswith(nii_exts)
])

# load images
def load_image(path: Path):
    if path.name.lower().endswith(nii_exts):
        img = nib.load(str(path))
        arr = img.get_fdata()
        # if this is a 3D volume, you might want to decide how to handle it—
        # here we assume 2D or single-slice
        return np.asarray(arr, dtype=np.float32)
    else:
        return imread(str(path)).astype(np.float32)

# load images
test_data = [load_image(f) for f in img_files]

# load ground‐truth if available
if gt_dir.exists():
    mask_files = sorted([
        f for f in gt_dir.iterdir()
        if f.suffix.lower() in img_exts
           or f.name.lower().endswith(nii_exts)
    ])
    test_labels = [load_image(f) for f in mask_files]
else:
    test_labels = None

# initialize your trained Cellpose model
model = models.CellposeModel(gpu=True, 
                            pretrained_model=trained_model)

# run inference
masks, flows, styles = model.eval(test_data, batch_size=32)

### # evaluate if ground‐truth was provided
### if test_labels is not None:
###     ap = metrics.average_precision(test_labels, masks)[0]
###     mean_ap50 = ap[:,0].mean()
###     print(f">>> average precision at IoU=0.5 : {mean_ap50:.3f}\n")

# save out each predicted mask
for img_path, mask in zip(img_files, masks):
    out_path = out_dir / f"{img_path.stem}_mask.tif"
    # cast to uint16 (or uint8) as needed
    imwrite(str(out_path), mask.astype(np.uint16))

print(f"Saved {len(masks)} predicted masks to:\n  {out_dir}")


plot masks

In [ ]:
import numpy as np
from pathlib import Path
from tifffile import imread, imwrite
import nibabel as nib
from cellpose import models, metrics


# path to the folder containing test images
test_dir = Path("/scratch/msa6093/OverlappingPatches-Y/s3")

# (optional) path to the folder containing ground‐truth masks
gt_dir   = Path("/path/to/your/gt_masks")

# where to save the predicted masks
out_dir  = test_dir / "/scratch/msa6093/Full_Masks_Overlapping_Y/s3"

# trained Cellpose model file
trained_model = "/scratch/msa6093/Daphnia392/models/CuratedTrainingData_May28_2Axes"




# make sure output directory exists
out_dir.mkdir(exist_ok=True, parents=True)

# gather all image files in test_dir
img_exts = {".tif", ".tiff", ".png", ".jpg", ".jpeg"}
nii_exts = (".nii", ".nii.gz")

# gather all image files in test_dir
img_files = sorted([
    f for f in test_dir.iterdir()
    if f.suffix.lower() in img_exts
       or f.name.lower().endswith(nii_exts)
])

# load images
def load_image(path: Path):
    if path.name.lower().endswith(nii_exts):
        img = nib.load(str(path))
        arr = img.get_fdata()
        # if this is a 3D volume, you might want to decide how to handle it—
        # here we assume 2D or single-slice
        return np.asarray(arr, dtype=np.float32)
    else:
        return imread(str(path)).astype(np.float32)

# load images
test_data = [load_image(f) for f in img_files]

# load ground‐truth if available
if gt_dir.exists():
    mask_files = sorted([
        f for f in gt_dir.iterdir()
        if f.suffix.lower() in img_exts
           or f.name.lower().endswith(nii_exts)
    ])
    test_labels = [load_image(f) for f in mask_files]
else:
    test_labels = None

# initialize your trained Cellpose model
model = models.CellposeModel(gpu=True, 
                            pretrained_model=trained_model)

# run inference
masks, flows, styles = model.eval(test_data, batch_size=32)

### # evaluate if ground‐truth was provided
### if test_labels is not None:
###     ap = metrics.average_precision(test_labels, masks)[0]
###     mean_ap50 = ap[:,0].mean()
###     print(f">>> average precision at IoU=0.5 : {mean_ap50:.3f}\n")

# save out each predicted mask
for img_path, mask in zip(img_files, masks):
    out_path = out_dir / f"{img_path.stem}_mask.tif"
    # cast to uint16 (or uint8) as needed
    imwrite(str(out_path), mask.astype(np.uint16))

print(f"Saved {len(masks)} predicted masks to:\n  {out_dir}")


In [ ]:
import numpy as np
from pathlib import Path
from tifffile import imread, imwrite
import nibabel as nib
from cellpose import models, metrics


# path to the folder containing test images
test_dir = Path("/scratch/msa6093/OverlappingPatches-Y/s4")

# (optional) path to the folder containing ground‐truth masks
gt_dir   = Path("/path/to/your/gt_masks")

# where to save the predicted masks
out_dir  = test_dir / "/scratch/msa6093/Full_Masks_Overlapping_Y/s4"

# trained Cellpose model file
trained_model = "/scratch/msa6093/Daphnia392/models/CuratedTrainingData_May28_2Axes"




# make sure output directory exists
out_dir.mkdir(exist_ok=True, parents=True)

# gather all image files in test_dir
img_exts = {".tif", ".tiff", ".png", ".jpg", ".jpeg"}
nii_exts = (".nii", ".nii.gz")

# gather all image files in test_dir
img_files = sorted([
    f for f in test_dir.iterdir()
    if f.suffix.lower() in img_exts
       or f.name.lower().endswith(nii_exts)
])

# load images
def load_image(path: Path):
    if path.name.lower().endswith(nii_exts):
        img = nib.load(str(path))
        arr = img.get_fdata()
        # if this is a 3D volume, you might want to decide how to handle it—
        # here we assume 2D or single-slice
        return np.asarray(arr, dtype=np.float32)
    else:
        return imread(str(path)).astype(np.float32)

# load images
test_data = [load_image(f) for f in img_files]

# load ground‐truth if available
if gt_dir.exists():
    mask_files = sorted([
        f for f in gt_dir.iterdir()
        if f.suffix.lower() in img_exts
           or f.name.lower().endswith(nii_exts)
    ])
    test_labels = [load_image(f) for f in mask_files]
else:
    test_labels = None

# initialize your trained Cellpose model
model = models.CellposeModel(gpu=True, 
                            pretrained_model=trained_model)

# run inference
masks, flows, styles = model.eval(test_data, batch_size=32)

### # evaluate if ground‐truth was provided
### if test_labels is not None:
###     ap = metrics.average_precision(test_labels, masks)[0]
###     mean_ap50 = ap[:,0].mean()
###     print(f">>> average precision at IoU=0.5 : {mean_ap50:.3f}\n")

# save out each predicted mask
for img_path, mask in zip(img_files, masks):
    out_path = out_dir / f"{img_path.stem}_mask.tif"
    # cast to uint16 (or uint8) as needed
    imwrite(str(out_path), mask.astype(np.uint16))

print(f"Saved {len(masks)} predicted masks to:\n  {out_dir}")


In [ ]:
import numpy as np
from pathlib import Path
from tifffile import imread, imwrite
import nibabel as nib
from cellpose import models, metrics


# path to the folder containing test images
test_dir = Path("/scratch/msa6093/OverlappingPatches-Y/s5")

# (optional) path to the folder containing ground‐truth masks
gt_dir   = Path("/path/to/your/gt_masks")

# where to save the predicted masks
out_dir  = test_dir / "/scratch/msa6093/Full_Masks_Overlapping_Y/s5"

# trained Cellpose model file
trained_model = "/scratch/msa6093/Daphnia392/models/CuratedTrainingData_May28_2Axes"




# make sure output directory exists
out_dir.mkdir(exist_ok=True, parents=True)

# gather all image files in test_dir
img_exts = {".tif", ".tiff", ".png", ".jpg", ".jpeg"}
nii_exts = (".nii", ".nii.gz")

# gather all image files in test_dir
img_files = sorted([
    f for f in test_dir.iterdir()
    if f.suffix.lower() in img_exts
       or f.name.lower().endswith(nii_exts)
])

# load images
def load_image(path: Path):
    if path.name.lower().endswith(nii_exts):
        img = nib.load(str(path))
        arr = img.get_fdata()
        # if this is a 3D volume, you might want to decide how to handle it—
        # here we assume 2D or single-slice
        return np.asarray(arr, dtype=np.float32)
    else:
        return imread(str(path)).astype(np.float32)

# load images
test_data = [load_image(f) for f in img_files]

# load ground‐truth if available
if gt_dir.exists():
    mask_files = sorted([
        f for f in gt_dir.iterdir()
        if f.suffix.lower() in img_exts
           or f.name.lower().endswith(nii_exts)
    ])
    test_labels = [load_image(f) for f in mask_files]
else:
    test_labels = None

# initialize your trained Cellpose model
model = models.CellposeModel(gpu=True, 
                            pretrained_model=trained_model)

# run inference
masks, flows, styles = model.eval(test_data, batch_size=32)

### # evaluate if ground‐truth was provided
### if test_labels is not None:
###     ap = metrics.average_precision(test_labels, masks)[0]
###     mean_ap50 = ap[:,0].mean()
###     print(f">>> average precision at IoU=0.5 : {mean_ap50:.3f}\n")

# save out each predicted mask
for img_path, mask in zip(img_files, masks):
    out_path = out_dir / f"{img_path.stem}_mask.tif"
    # cast to uint16 (or uint8) as needed
    imwrite(str(out_path), mask.astype(np.uint16))

print(f"Saved {len(masks)} predicted masks to:\n  {out_dir}")


In [ ]:
plt.figure(figsize=(12,8), dpi=150)
for k,im in enumerate(test_data):
    img = im.copy()
    plt.subplot(3,len(test_data), k+1)
    img = np.vstack((img, np.zeros_like(img)[:1]))
    img = img.transpose(1,2,0)
    plt.imshow(img)
    plt.axis('off')
    if k==0:
        plt.title('image')

    plt.subplot(3,len(test_data), len(test_data) + k+1)
    plt.imshow(masks[k])
    plt.axis('off')
    if k==0:
        plt.title('predicted labels')

    plt.subplot(3,len(test_data), 2*len(test_data) + k+1)
    plt.imshow(test_labels[k])
    plt.axis('off')
    if k==0:
        plt.title('true labels')
plt.tight_layout()